In [1]:
import json
import requests
import pandas as pd
import time

In [2]:
kobe={}
kobe["shot_loc"]={}
kobe["shot_stat"]={}
kobe["season_list"]=["1996-97","1997-98","1998-99","1999-00","2000-01","2001-02","2002-03","2003-04",
                     "2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12",
                    "2012-13","2013-14","2014-15","2015-16"]
#kobe["playoff"]=[]
#kobe["teammate"]={}
#kobe["champion"]={}

In [ ]:
#kobe["teammate"]=[""]

In [3]:
url_list=[]
for item in kobe["season_list"]:
    url='http://stats.nba.com/stats/shotchartdetail?Period=0&VsConference=&LeagueID=00&LastNGames=0&TeamID=0&Position=&Location=&Outcome=&ContextMeasure=FGA&DateFrom=&StartPeriod=&DateTo=&OpponentTeamID=0&ContextFilter=&RangeType=&Season='+item+'&AheadBehind=&PlayerID=977&EndRange=&VsDivision=&PointDiff=&RookieYear=&GameSegment=&Month=0&ClutchTime=&StartRange=&EndPeriod=&SeasonType=Regular+Season&SeasonSegment=&GameID=&PlayerPosition='
    url_list.append(url)
shot_chart_url = 'http://stats.nba.com/stats/shotchartdetail?Period=0&VsConference=&LeagueID=00&LastNGames=0&TeamID=0&Position=&Location=&Outcome=&ContextMeasure=FGA&DateFrom=&StartPeriod=&DateTo=&OpponentTeamID=0&ContextFilter=&RangeType=&Season=2014-15&AheadBehind=&PlayerID=977&EndRange=&VsDivision=&PointDiff=&RookieYear=&GameSegment=&Month=0&ClutchTime=&StartRange=&EndPeriod=&SeasonType=Regular+Season&SeasonSegment=&GameID=&PlayerPosition='

# Get the webpage containing the data
header = requests.utils.default_headers()
header.update(
    {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    }
)

In [4]:
for index in range(len(url_list)):
    response = requests.get(url_list[index],headers=header)
    # Grab the headers to be used as column headers for our DataFrame
    headers = response.json()['resultSets'][0]['headers']
    # Grab the shot chart data
    shots = response.json()['resultSets'][0]['rowSet']
    
    shot_df = pd.DataFrame(shots, columns=headers)
    
    kobe['shot_loc'][kobe['season_list'][index]]=[]
    shot_att=[0 for _ in range(31)]
    shot_made=[0 for _ in range(31)]
    kobe["shot_stat"][kobe['season_list'][index]]=[]
    
    for j in range(len(shot_df)):
        if shot_df['SHOT_DISTANCE'][j]>30:
            continue
        shot_loc={}
        shot_loc['x']=(shot_df['LOC_X'][j]+250)/10.0
        shot_loc['y']=(shot_df['LOC_Y'][j]+47.5)/10.0

#        shot_loc['action_type']=shot_df['SHOT_TYPE'][j].encode('utf8')[:4]+shot_df['ACTION_TYPE'][j].encode('utf8')
        shot_loc['shot_distance']=shot_df['SHOT_DISTANCE'][j]
        shot_loc['shot_made_flag']=shot_df['SHOT_MADE_FLAG'][j]
        kobe['shot_loc'][kobe['season_list'][index]].append(shot_loc)
        if shot_df['SHOT_DISTANCE'][j]<31:
            shot_att[shot_df['SHOT_DISTANCE'][j]]+=1
            if shot_df['SHOT_MADE_FLAG'][j]==1:
                shot_made[shot_df['SHOT_DISTANCE'][j]]+=1
                
    for j in range(len(shot_att)):
        shot_stat={}
        shot_stat['dis']=j
        shot_stat['freq']=1.0*shot_att[j]/sum(shot_att)
 
        if shot_att[j]>0:
            shot_stat['percentage']=1.0*shot_made[j]/shot_att[j]
        else:
            shot_stat['percentage']=0.0        
        kobe["shot_stat"][kobe['season_list'][index]].append(shot_stat)
        
    time.sleep(1)

In [5]:
json_str=json.dumps(kobe)
with open('test.json', 'w') as f:
    json.dump(json_str, f)

In [6]:
for item in kobe["shot_loc"]:
    for loc in kobe["shot_loc"][item]:
        if loc["shot_distance"]>30:
            print loc["shot_distance"]